In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import seaborn as sns
from datetime import datetime, date, timedelta
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

In [2]:
df = pd.read_excel(r'C:\Users\basia\OneDrive\Pulpit\sdu shit\thesis\COUPON_REPORT-2022.xlsx')
df.head()


,Ticket Number,Issue date,PAX first name,PAX last name,PAX type,Origin,Destination,Departure Date,Departure Time,Arrival Date,Arrival Time,Fare Basis Code,SMS Total Org,GDS Total Org,Org Issuer Company,Org Distribution Channel
0,2482100342283,2019-12-04,Lucas,van der Laan,Adult,SGD,IOM,2022-05-19,07:45,2022-05-19,09:30,HWENDPLUS,1049,0,Ecommerce,6I
1,2482100342283,2019-12-04,Lucas,van der Laan,Adult,IOM,SGD,2022-05-22,15:00,2022-05-22,18:45,HWENDPLUS,1049,0,Ecommerce,6I
2,2482100342284,2019-12-04,Anne Mette,van der Laan,Adult,SGD,IOM,2022-05-19,07:45,2022-05-19,09:30,HWENDBASIC,949,0,Ecommerce,6I
3,2482100342284,2019-12-04,Anne Mette,van der Laan,Adult,IOM,SGD,2022-05-22,15:00,2022-05-22,18:45,HWENDBASIC,949,0,Ecommerce,6I
4,2482100342291,2019-12-04,Andreas Peter,Rebsdorf,Adult,SGD,IOM,2022-05-19,07:45,2022-05-19,09:30,MWENDPLUS,1299,0,Ecommerce,6I


In [3]:
df.shape

(45032, 16)

In [4]:
#Focusing only on Sonderborg-Copenhagen or Copenhagen-Sonderborg 

options = ['SGD', 'CPH'] 
    
# selecting rows based on condition 
df = df[df['Origin'].isin(options)] 
df = df[df['Destination'].isin(options)] 
df.shape

(38343, 16)

In [5]:
#Type of flight (only for e-commerce)
df['Fare Basis Code cut']=df['Fare Basis Code'].str[1:4]
df['Full Name']=df["PAX first name"]+" "+df["PAX last name"]
df['Full Name']=df['Full Name'].str.lower()
df.head()

,Ticket Number,Issue date,PAX first name,PAX last name,PAX type,Origin,Destination,Departure Date,Departure Time,Arrival Date,Arrival Time,Fare Basis Code,SMS Total Org,GDS Total Org,Org Issuer Company,Org Distribution Channel,Fare Basis Code cut,Full Name
188,2482100362470,2021-07-16,Jens Chr,Jensen,Adult,CPH,SGD,2022-03-20,19:00,2022-03-20,19:45,LPLUOW,749,0,Ecommerce,6I,PLU,jens chr jensen
189,2482100362471,2021-07-16,Dorthe Vendelbo,Jensen,Adult,CPH,SGD,2022-03-20,19:00,2022-03-20,19:45,LPLUOW,749,0,Ecommerce,6I,PLU,dorthe vendelbo jensen
198,2482100364603,2021-08-13,Thomas,Dehnfeld,Adult,SGD,CPH,2022-03-09,07:00,2022-03-09,07:45,WPLURT,749,0,Ecommerce,6I,PLU,thomas dehnfeld
199,2482100364603,2021-08-13,Thomas,Dehnfeld,Adult,CPH,SGD,2022-03-15,18:00,2022-03-15,18:45,WPLURT,749,0,Ecommerce,6I,PLU,thomas dehnfeld
200,2482100364604,2021-08-13,Jane,Dehnfeld,Adult,SGD,CPH,2022-03-09,07:00,2022-03-09,07:45,WPLURT,749,0,Ecommerce,6I,PLU,jane dehnfeld


In [7]:
dfr = pd.read_excel(r'C:\Users\basia\OneDrive\Pulpit\sdu shit\thesis\Reservations_2022.xlsx')
dfr.head()

,eTicket number,Booking creation time,Passenger first name,Passenger last name,Contact person email,Unnamed: 5,Departure local date
0,2482100394150,2022-10-05 12:09:04,ANNA,MUUS,pnfimport121@worldticket.net,NaN,2022-10-08
1,2486939062643,2022-03-03 15:44:02,CLAUS,PETERSEN,CLAUS.PETERSEN@DANFOSS.COM,NaN,2022-03-21
2,2482100379154,2022-03-10 06:56:59,Cecilie,Kroigaard,cil@alsie.com,NaN,2022-03-11
3,1055262232464,2022-10-08 07:34:24,ANTTI,SUMMANEN,ANTTI.SUMMANEN@DANFOSS.COM,NaN,2022-10-11
4,2482100394507,2022-10-10 04:37:21,TRINE,FISCHER,Cecilie10lorenzen@gmail.com,NaN,2022-12-02


In [8]:
del dfr['Unnamed: 5']
del dfr ['Departure local date']


In [9]:
dfr.dtypes

eTicket number            int64
Booking creation time    object
Passenger first name     object
Passenger last name      object
Contact person email     object
dtype: object

In [10]:
#Extracting type of travel based on email 

dfr2=dfr.copy()

#domains=['@hotmail','@gmail','@yahoo', '@aol', '@msn', '@wanadoo', '@orange', '@comcast', '@live', '@gmx']
dfr2['Contact person email']=dfr2['Contact person email'].str.contains('@hotmail|@gmail|@yahoo|@aol|@msn|@wanadoo|@orange|@comcast|@live|@gmx')
#dfr2['Passenger email']=dfr2['Passenger email'].str.contains('@hotmail|@gmail|@yahoo|@aol|@msn|@wanadoo|@orange|@comcast|@live|@gmx')
dfr['Type']=dfr2['Contact person email']
dfr['Type'] = dfr['Type'].map({True: 'Leisure', False: 'Business'}) 
dfr.head()

,eTicket number,Booking creation time,Passenger first name,Passenger last name,Contact person email,Type
0,2482100394150,2022-10-05 12:09:04,ANNA,MUUS,pnfimport121@worldticket.net,Business
1,2486939062643,2022-03-03 15:44:02,CLAUS,PETERSEN,CLAUS.PETERSEN@DANFOSS.COM,Business
2,2482100379154,2022-03-10 06:56:59,Cecilie,Kroigaard,cil@alsie.com,Business
3,1055262232464,2022-10-08 07:34:24,ANTTI,SUMMANEN,ANTTI.SUMMANEN@DANFOSS.COM,Business
4,2482100394507,2022-10-10 04:37:21,TRINE,FISCHER,Cecilie10lorenzen@gmail.com,Leisure


In [11]:
dfr.shape

(45934, 6)

In [12]:
dfr=dfr.rename(columns = {'eTicket number':'Ticket Number'})
dfr.head()

,Ticket Number,Booking creation time,Passenger first name,Passenger last name,Contact person email,Type
0,2482100394150,2022-10-05 12:09:04,ANNA,MUUS,pnfimport121@worldticket.net,Business
1,2486939062643,2022-03-03 15:44:02,CLAUS,PETERSEN,CLAUS.PETERSEN@DANFOSS.COM,Business
2,2482100379154,2022-03-10 06:56:59,Cecilie,Kroigaard,cil@alsie.com,Business
3,1055262232464,2022-10-08 07:34:24,ANTTI,SUMMANEN,ANTTI.SUMMANEN@DANFOSS.COM,Business
4,2482100394507,2022-10-10 04:37:21,TRINE,FISCHER,Cecilie10lorenzen@gmail.com,Leisure


In [13]:
dfr['Booking creation time']=pd.to_datetime(dfr['Booking creation time'])
#dfr['Departure Date']=pd.to_datetime(dfr['Departure Date'])
#from string to date
dfr['Booking creation day']=pd.to_datetime(dfr['Booking creation time']).dt.date
dfr['Booking Hour']=pd.to_datetime(dfr['Booking creation time']).dt.hour


In [18]:
#combining names to create final merge

#dfr['Days before']=dfr['Departure Date']-dfr['Booking creation time']
dfr['Full Name']=dfr["Passenger first name"]+" "+dfr["Passenger last name"]
dfr['Full Name']=dfr['Full Name'].str.lower()
dfr.head()

,Ticket Number,Booking creation time,Passenger first name,Passenger last name,Contact person email,Type,Booking creation day,Booking Hour,Full Name
0,2482100394150,2022-10-05 12:09:04,ANNA,MUUS,pnfimport121@worldticket.net,Business,2022-10-05,12,anna muus
1,2486939062643,2022-03-03 15:44:02,CLAUS,PETERSEN,CLAUS.PETERSEN@DANFOSS.COM,Business,2022-03-03,15,claus petersen
2,2482100379154,2022-03-10 06:56:59,Cecilie,Kroigaard,cil@alsie.com,Business,2022-03-10,6,cecilie kroigaard
3,1055262232464,2022-10-08 07:34:24,ANTTI,SUMMANEN,ANTTI.SUMMANEN@DANFOSS.COM,Business,2022-10-08,7,antti summanen
4,2482100394507,2022-10-10 04:37:21,TRINE,FISCHER,Cecilie10lorenzen@gmail.com,Leisure,2022-10-10,4,trine fischer


In [19]:
dfr.dtypes

Ticket Number                     int64
Booking creation time    datetime64[ns]
Passenger first name             object
Passenger last name              object
Contact person email             object
Type                             object
Booking creation day             object
Booking Hour                      int64
Full Name                        object
dtype: object

In [20]:
df.dtypes

Ticket Number                int64
Issue date                  object
PAX first name              object
PAX last name               object
PAX type                    object
Origin                      object
Destination                 object
Departure Date              object
Departure Time              object
Arrival Date                object
Arrival Time                object
Fare Basis Code             object
SMS Total Org               object
GDS Total Org               object
Org Issuer Company          object
Org Distribution Channel    object
Fare Basis Code cut         object
Full Name                   object
dtype: object

In [21]:
df['Ticket Number'] = df['Ticket Number'].astype('int64', copy=False)
df.dtypes
df.head()

,Ticket Number,Issue date,PAX first name,PAX last name,PAX type,Origin,Destination,Departure Date,Departure Time,Arrival Date,Arrival Time,Fare Basis Code,SMS Total Org,GDS Total Org,Org Issuer Company,Org Distribution Channel,Fare Basis Code cut,Full Name
188,2482100362470,2021-07-16,Jens Chr,Jensen,Adult,CPH,SGD,2022-03-20,19:00,2022-03-20,19:45,LPLUOW,749,0,Ecommerce,6I,PLU,jens chr jensen
189,2482100362471,2021-07-16,Dorthe Vendelbo,Jensen,Adult,CPH,SGD,2022-03-20,19:00,2022-03-20,19:45,LPLUOW,749,0,Ecommerce,6I,PLU,dorthe vendelbo jensen
198,2482100364603,2021-08-13,Thomas,Dehnfeld,Adult,SGD,CPH,2022-03-09,07:00,2022-03-09,07:45,WPLURT,749,0,Ecommerce,6I,PLU,thomas dehnfeld
199,2482100364603,2021-08-13,Thomas,Dehnfeld,Adult,CPH,SGD,2022-03-15,18:00,2022-03-15,18:45,WPLURT,749,0,Ecommerce,6I,PLU,thomas dehnfeld
200,2482100364604,2021-08-13,Jane,Dehnfeld,Adult,SGD,CPH,2022-03-09,07:00,2022-03-09,07:45,WPLURT,749,0,Ecommerce,6I,PLU,jane dehnfeld


In [22]:
merged_df = pd.merge(df,dfr, how = 'inner', on = ['Ticket Number', 'Full Name'])

In [164]:
merged_df.head()

,Ticket Number,Issue date,PAX first name,PAX last name,PAX type,Origin,Destination,Departure Time,Arrival Date,Arrival Time,...,Fare Basis Code cut,Full Name,Booking creation time,Passenger first name,Passenger last name,Contact person email,Departure Date,Type,Booking creation day,Booking Hour
0,2482100362471,2021-07-16,Dorthe Vendelbo,Jensen,Adult,CPH,SGD,19:00,2022-03-20,19:45,...,PLU,dorthe vendelbo jensen,2021-07-16 17:24:10,Dorthe Vendelbo,Jensen,vendel@stofanet.dk,2022-03-20,Business,2021-07-16,17
1,2482100364603,2021-08-13,Thomas,Dehnfeld,Adult,SGD,CPH,07:00,2022-03-09,07:45,...,PLU,thomas dehnfeld,2021-08-13 12:52:32,Thomas,Dehnfeld,td@kf17.dk,2022-03-09,Business,2021-08-13,12
2,2482100364603,2021-08-13,Thomas,Dehnfeld,Adult,SGD,CPH,07:00,2022-03-09,07:45,...,PLU,thomas dehnfeld,2021-08-13 12:52:32,Thomas,Dehnfeld,td@kf17.dk,2022-03-15,Business,2021-08-13,12
3,2482100364603,2021-08-13,Thomas,Dehnfeld,Adult,CPH,SGD,18:00,2022-03-15,18:45,...,PLU,thomas dehnfeld,2021-08-13 12:52:32,Thomas,Dehnfeld,td@kf17.dk,2022-03-09,Business,2021-08-13,12
4,2482100364603,2021-08-13,Thomas,Dehnfeld,Adult,CPH,SGD,18:00,2022-03-15,18:45,...,PLU,thomas dehnfeld,2021-08-13 12:52:32,Thomas,Dehnfeld,td@kf17.dk,2022-03-15,Business,2021-08-13,12


In [23]:
del merged_df['Passenger first name'], merged_df['Passenger last name'], merged_df['PAX first name'], merged_df['PAX last name']

In [24]:
merged_df.head()

,Ticket Number,Issue date,PAX type,Origin,Destination,Departure Date,Departure Time,Arrival Date,Arrival Time,Fare Basis Code,...,GDS Total Org,Org Issuer Company,Org Distribution Channel,Fare Basis Code cut,Full Name,Booking creation time,Contact person email,Type,Booking creation day,Booking Hour
0,2482100362471,2021-07-16,Adult,CPH,SGD,2022-03-20,19:00,2022-03-20,19:45,LPLUOW,...,0,Ecommerce,6I,PLU,dorthe vendelbo jensen,2021-07-16 17:24:10,vendel@stofanet.dk,Business,2021-07-16,17
1,2482100364603,2021-08-13,Adult,SGD,CPH,2022-03-09,07:00,2022-03-09,07:45,WPLURT,...,0,Ecommerce,6I,PLU,thomas dehnfeld,2021-08-13 12:52:32,td@kf17.dk,Business,2021-08-13,12
2,2482100364603,2021-08-13,Adult,SGD,CPH,2022-03-09,07:00,2022-03-09,07:45,WPLURT,...,0,Ecommerce,6I,PLU,thomas dehnfeld,2021-08-13 12:52:32,td@kf17.dk,Business,2021-08-13,12
3,2482100364603,2021-08-13,Adult,CPH,SGD,2022-03-15,18:00,2022-03-15,18:45,WPLURT,...,0,Ecommerce,6I,PLU,thomas dehnfeld,2021-08-13 12:52:32,td@kf17.dk,Business,2021-08-13,12
4,2482100364603,2021-08-13,Adult,CPH,SGD,2022-03-15,18:00,2022-03-15,18:45,WPLURT,...,0,Ecommerce,6I,PLU,thomas dehnfeld,2021-08-13 12:52:32,td@kf17.dk,Business,2021-08-13,12


In [25]:
merged_df.shape#solution created doubled duplicated rows 
merged_df=merged_df.drop_duplicates()
print(merged_df.shape)
merged_df.head()

(33473, 21)


,Ticket Number,Issue date,PAX type,Origin,Destination,Departure Date,Departure Time,Arrival Date,Arrival Time,Fare Basis Code,...,GDS Total Org,Org Issuer Company,Org Distribution Channel,Fare Basis Code cut,Full Name,Booking creation time,Contact person email,Type,Booking creation day,Booking Hour
0,2482100362471,2021-07-16,Adult,CPH,SGD,2022-03-20,19:00,2022-03-20,19:45,LPLUOW,...,0,Ecommerce,6I,PLU,dorthe vendelbo jensen,2021-07-16 17:24:10,vendel@stofanet.dk,Business,2021-07-16,17
1,2482100364603,2021-08-13,Adult,SGD,CPH,2022-03-09,07:00,2022-03-09,07:45,WPLURT,...,0,Ecommerce,6I,PLU,thomas dehnfeld,2021-08-13 12:52:32,td@kf17.dk,Business,2021-08-13,12
3,2482100364603,2021-08-13,Adult,CPH,SGD,2022-03-15,18:00,2022-03-15,18:45,WPLURT,...,0,Ecommerce,6I,PLU,thomas dehnfeld,2021-08-13 12:52:32,td@kf17.dk,Business,2021-08-13,12
5,2482100364604,2021-08-13,Adult,SGD,CPH,2022-03-09,07:00,2022-03-09,07:45,WPLURT,...,0,Ecommerce,6I,PLU,jane dehnfeld,2021-08-13 12:52:32,td@kf17.dk,Business,2021-08-13,12
7,2482100364604,2021-08-13,Adult,CPH,SGD,2022-03-15,18:00,2022-03-15,18:45,WPLURT,...,0,Ecommerce,6I,PLU,jane dehnfeld,2021-08-13 12:52:32,td@kf17.dk,Business,2021-08-13,12


In [26]:
merged_df.to_excel(r'C:\Users\basia\OneDrive\Pulpit\sdu shit\thesis\READY DATASETS\merged2022.xlsx', index=False)